In [1]:
import pandas as pd

In [2]:
frame = pd.read_csv('10x10, 10^5 images')

In [3]:
frame.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0,1,1,0,0,0,1,0,1,1,...,1,1,1,0,0,0,0,1,1,8
1,1,0,1,1,1,0,0,1,1,1,...,1,1,0,1,1,1,0,1,0,6
2,1,1,1,1,0,1,0,0,1,1,...,0,0,0,1,0,1,1,0,0,6
3,0,0,0,1,0,0,0,0,0,0,...,1,1,0,1,0,0,1,1,1,7
4,1,0,1,1,1,1,0,0,1,1,...,1,0,1,0,0,0,0,1,1,10


In [4]:
print(frame['100'].mean(), frame['100'].median())

10.22975 10.0


In [5]:
# Собственно если совсем ничего не делать
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print(mean_squared_error(frame['100'], [frame['100'].mean()] * frame.shape[0]))
print(mean_absolute_error(frame['100'], [frame['100'].median()] * frame.shape[0]))

9.2851049375
2.42221


In [6]:
X = frame.drop('100', axis = 1)
y = frame['100']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Maybe KNN

In [9]:
#Собственно если сделать чуть-чуть
from sklearn.neighbors import KNeighborsRegressor
from sklearn.grid_search import GridSearchCV

%%time
knn = KNeighborsRegressor()
param_grid = {'n_neighbors': [1, 3, 5, 10, 30, 100, 1000], 'p': [1, 1.5, 2, 3]}
param_grid_mini = {'n_neighbors': [5], 'p': [1]}
optimizer = GridSearchCV(knn, param_grid_mini, scoring='neg_mean_squared_error', n_jobs=-1, cv = 2)
optimizer.fit(X_train, y_train)

In [16]:
%%time
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

CPU times: user 11.5 s, sys: 24 ms, total: 11.5 s
Wall time: 11.6 s


In [17]:
%%time
print(mean_squared_error(y_test[1000:2000], knn.predict(X_test[1000:2000])))

7.38924
CPU times: user 24.4 s, sys: 12 ms, total: 24.4 s
Wall time: 24.5 s
